### Amazon Route53 Resolver Query Logs
##### Read CSV to DataFrame

In [ ]:
import pandas as pd

df = pd.read_csv('/tmp/68eff94b-a283-48e7-93af-42edfbda8e2a.csv', sep=',')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df.head(1)

##### Identify Query Source

In [ ]:
pd.crosstab(
    index = [
        df["query_hostname"],
        df["query_type"],
        df["accountid"],
        df["region"],
        df["src_vpc_uid"],
        df["src_instance_uid"],
        df["src_ip"]
    ], 
    columns = [
        df["rcode"]
    ],                      
    margins = True
).sort_index(
    axis = 1,
    ascending = False
).sort_values(
    by = [
        'All'
    ],
    ascending = False
)

##### Identify Host Source

In [ ]:
pd.crosstab(
    index = [
        df["src_ip"],
        df["src_instance_uid"],
        df["src_vpc_uid"],
        df["region"],
        df["accountid"],
        df["query_type"],
        df["query_hostname"]
    ], 
    columns = [
        df["rcode"]
    ],                      
    margins = True
).sort_index(
    axis = 1,
    ascending = False
).sort_values(
    by = [
        'All'
    ],
    ascending = False
)

##### Query Times

In [ ]:
import matplotlib.pyplot as plt

df1 = df['time'].value_counts().rename_axis('time').reset_index(name='counts')
df1 = df1.sort_values(by=['time'], ascending=True)

plt.figure(figsize=(20,10))
plt.scatter(df1['time'], df1['counts'])
plt.show()

##### Query Deviations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

df2 = df['query_hostname'].value_counts().rename_axis('query_hostname').reset_index(name='counts')

records = df2.loc[:,'counts'].to_numpy()
mean = records.mean()
median = np.median(records)
stddev = np.std(records)

plt.figure(figsize=(20,10))
plt.hist(df2.loc[:,'counts'], log=True)
plt.plot([mean, mean],[0,15**4], label="Mean")
plt.plot([mean+stddev,mean+stddev],[0,15**4], label="+1 Std")
plt.plot([mean+(stddev*2),mean+(stddev*2)],[0,15**4], label="+2 Std")

plt.legend()
plt.show()

##### Identify Query Answers

In [ ]:
df3 = df[['query_hostname', 'rcode', 'answers']]
df3 = df3.dropna()

items = []
number = 0

for index, row in df3.iterrows():
    try:
        answers = row['answers'].split('}, {')
        for answer in answers:
            answer = answer.replace('[','').replace(']','').replace('{','').replace('}','')
            item = answer.split(', ')
            items.append([row['query_hostname'],row['rcode'],item[1][6:]])
    except:
        number += 1
        pass

print('Query Answers: '+str(len(items)))
print('No Query Answers: '+str(number))

df4 = pd.DataFrame(items, columns = ['query_hostname', 'rcode', 'answer'])

pd.crosstab(
    index = [
        df4["query_hostname"],
        df4["answer"]
    ], 
    columns = [
        df4["rcode"]
    ],                      
    margins = True
).sort_index(
    axis = 1,
    ascending = False
).sort_values(
    by = [
        'All'
    ],
    ascending = False
)

##### Answer Deviations

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

df5 = df4['answer'].value_counts().rename_axis('answer').reset_index(name='counts')

records = df5.loc[:,'counts'].to_numpy()
mean = records.mean()
median = np.median(records)
stddev = np.std(records)

plt.figure(figsize=(20,10))
plt.hist(df5.loc[:,'counts'], log=True)
plt.plot([mean, mean],[0,15**4], label="Mean")
plt.plot([mean+stddev,mean+stddev],[0,15**4], label="+1 Std")
plt.plot([mean+(stddev*2),mean+(stddev*2)],[0,15**4], label="+2 Std")

plt.legend()
plt.show()